**📘 Introducción**





En esta tarea se implementan y aplican tres algoritmos importantes:

*  K-Means: para segmentación de clientes utilizando un dataset.
*  
Kruskal y Prim: algoritmos clásicos de grafos para encontrar árboles de expansión mínima (MST).

**🔶 K-Means con Dataset de Customer Segmentation**


Implementación de K-Means en C++
El algoritmo de K-Means agrupa los puntos en K clústeres según su distancia a centroides.

In [22]:
%%writefile kmeans.cpp
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <cmath>
#include <cstdlib>
#include <ctime>
#include <limits>

using namespace std;

class Point
{
public:
    vector<double> features;
    int cluster_id;

    Point(const vector<double> &f) : features(f), cluster_id(-1) {}
};

class KMeans
{
private:
    int k;
    int max_iters;
    vector<vector<double>> centroids;

    double euclideanDistance(const vector<double> &a, const vector<double> &b)
    {
        double sum = 0.0;
        for (size_t i = 0; i < a.size(); ++i)
            sum += pow(a[i] - b[i], 2);
        return sqrt(sum);
    }

    void assignClusters(vector<Point> &points)
    {
        for (auto &point : points)
        {
            double min_dist = numeric_limits<double>::max();
            int best_cluster = -1;
            for (int i = 0; i < k; ++i)
            {
                double dist = euclideanDistance(point.features, centroids[i]);
                if (dist < min_dist)
                {
                    min_dist = dist;
                    best_cluster = i;
                }
            }
            point.cluster_id = best_cluster;
        }
    }

    void updateCentroids(vector<Point> &points)
    {
        vector<vector<double>> new_centroids(k, vector<double>(points[0].features.size(), 0.0));
        vector<int> counts(k, 0);

        for (const auto &point : points)
        {
            for (size_t j = 0; j < point.features.size(); ++j)
                new_centroids[point.cluster_id][j] += point.features[j];
            counts[point.cluster_id]++;
        }

        for (int i = 0; i < k; ++i)
        {
            if (counts[i] == 0)
                continue;
            for (size_t j = 0; j < new_centroids[i].size(); ++j)
                new_centroids[i][j] /= counts[i];
        }

        centroids = new_centroids;
    }

public:
    KMeans(int k, int max_iters) : k(k), max_iters(max_iters) {}

    void fit(vector<Point> &points)
    {
        centroids.clear();
        srand(time(0));
        for (int i = 0; i < k; ++i)
        {
            int index = rand() % points.size();
            centroids.push_back(points[index].features);
        }

        for (int iter = 0; iter < max_iters; ++iter)
        {
            assignClusters(points);
            updateCentroids(points);
        }
    }

    void printResults(const vector<Point> &points)
    {
        vector<int> cluster_counts(k, 0);

        for (const auto &point : points)
        {
            cluster_counts[point.cluster_id]++;
            cout << "Cluster " << point.cluster_id << ": ";
            for (double val : point.features)
                cout << val << " ";
            cout << endl;
        }

        cout << "\n--- Resumen por Clúster ---\n";
        for (int i = 0; i < k; ++i)
        {
            cout << "Clúster " << i << ": " << cluster_counts[i] << " elementos\n";
        }

        cout << "\n--- Centroides Finales ---\n";
        for (int i = 0; i < k; ++i)
        {
            cout << "Clúster " << i << " centroide: ";
            for (double val : centroids[i])
                cout << val << " ";
            cout << endl;
        }
    }
};

vector<Point> loadData(const string &filename)
{
    ifstream file(filename);
    string line;
    vector<Point> data;

    bool header_skipped = false;

    while (getline(file, line))
    {
        if (!header_skipped)
        {
            header_skipped = true;
            continue;
        }

        stringstream ss(line);
        string cell;
        vector<double> features;
        int col = 0;

        while (getline(ss, cell, '\t'))
        {
            // Column indices: Income = 4, MntWines = 9 to MntGoldProds = 14
            if (col == 4 || (col >= 9 && col <= 14))
            {
                try
                {
                    double val = cell.empty() ? 0.0 : stod(cell);
                    features.push_back(val);
                }
                catch (...)
                {
                    features.push_back(0.0); // fallback for non-numeric or missing values
                }
            }
            col++;
        }

        if (!features.empty())
        {
            data.emplace_back(features);
        }
    }

    return data;
}

int main()
{
    vector<Point> data = loadData("kmeans_data.txt");
    if (data.empty())
    {
        cerr << "No data loaded. Verifica el archivo.\n";
        return 1;
    }

    KMeans kmeans(3, 100);
    kmeans.fit(data);
    kmeans.printResults(data);

    return 0;
}

Overwriting kmeans.cpp


In [23]:
!wget https://raw.githubusercontent.com/ThomasSan15/DataStructures/main/KmeansHw/kmeans_data.txt -O kmeans_data.txt


!g++ kmeans.cpp -o kmeans
!./kmeans


--2025-05-21 15:29:33--  https://raw.githubusercontent.com/ThomasSan15/DataStructures/main/KmeansHw/kmeans_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56426 (55K) [text/plain]
Saving to: ‘kmeans_data.txt’

kmeans_data.txt     100%[===================>]  55.10K  --.-KB/s    in 0.006s  

2025-05-21 15:29:33 (9.74 MB/s) - ‘kmeans_data.txt’ saved [56426/56426]

Cluster 2: 172 
Cluster 1: 2 
Cluster 0: 111 
Cluster 1: 10 
Cluster 0: 46 
Cluster 1: 0 
Cluster 0: 50 
Cluster 1: 3 
Cluster 1: 3 
Cluster 1: 1 
Cluster 1: 11 
Cluster 2: 225 
Cluster 1: 3 
Cluster 1: 6 
Cluster 0: 59 
Cluster 1: 2 
Cluster 2: 150 
Cluster 1: 0 
Cluster 1: 30 
Cluster 0: 69 
Cluster 1: 1 
Cluster 1: 0 
Cluster 1: 21 
Cluster 1: 39 
Cluster 1: 15 
Cluster 1: 3 
Cluster 1: 20

**🌐 Algoritmo de Kruskal**



**Descripción**

Kruskal busca un árbol de expansión mínima seleccionando aristas por orden de peso, evitando ciclos.

Estructuras clave:

Lista de aristas con pesos

Union-Find (Disjoint Set Union)

Implementación en C++


In [10]:
%%writefile kruskal.cpp
#include <iostream>
#include <vector>
#include <algorithm>
#include <random>
using namespace std;

class UnionFind {
private:
    vector<int> parent, rank;

public:
    UnionFind(int n) {
        parent.resize(n);
        rank.resize(n, 0);
        for (int i = 0; i < n; ++i)
            parent[i] = i;
    }

    int find(int x) {
        if (parent[x] != x)
            parent[x] = find(parent[x]);
        return parent[x];
    }

    bool union_sets(int x, int y) {
        int xr = find(x);
        int yr = find(y);
        if (xr == yr)
            return false;
        if (rank[xr] < rank[yr])
            parent[xr] = yr;
        else if (rank[xr] > rank[yr])
            parent[yr] = xr;
        else {
            parent[yr] = xr;
            rank[xr]++;
        }
        return true;
    }
};

class Graph {
private:
    vector<vector<int>> adj_matrix;
    vector<int> edges_u;
    vector<int> edges_v;
    vector<int> edges_weight;

public:
    Graph(int n) {
        adj_matrix = vector<vector<int>>(n, vector<int>(n, -1));
    }

    void add_edge(int u, int v, int weight) {
        adj_matrix[u][v] = weight;
        adj_matrix[v][u] = weight;
    }

    void print() {
        cout << "Matriz de adyacencia ponderada:\n";
        for (const auto& row : adj_matrix) {
            for (int val : row) {
                if (val == -1) cout << ". ";
                else cout << val << " ";
            }
            cout << endl;
        }
    }

    void extract_edges() {
        int n = adj_matrix.size();
        edges_u.clear();
        edges_v.clear();
        edges_weight.clear();

        for (int i = 0; i < n; ++i) {
            for (int j = i + 1; j < n; ++j) {
                if (adj_matrix[i][j] != -1) {
                    edges_u.push_back(i);
                    edges_v.push_back(j);
                    edges_weight.push_back(adj_matrix[i][j]);
                }
            }
        }
    }

    void kruskal() {
        extract_edges();
        int n = adj_matrix.size();
        int m = edges_u.size();


        vector<int> indices(m);
        for (int i = 0; i < m; ++i) indices[i] = i;

        sort(indices.begin(), indices.end(), [&](int a, int b) {
            return edges_weight[a] < edges_weight[b];
        });

        UnionFind uf(n);
        int mst_weight = 0;

        cout << "\nAristas del Árbol de Expansión Mínima (Kruskal):\n";
        for (int i : indices) {
            int u = edges_u[i];
            int v = edges_v[i];
            int w = edges_weight[i];

            if (uf.union_sets(u, v)) {
                cout << u << " - " << v << " (peso " << w << ")\n";
                mst_weight += w;
            }
        }
        cout << "Peso total del MST: " << mst_weight << endl;
    }
};

int main() {
    int n = 11;
    Graph g(n);

    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> distrib(0, n - 1);
    uniform_int_distribution<> weight_dist(1, 100);

    for (int i = 0; i < 10; i++) {
        int u = distrib(gen);
        int v = distrib(gen);
        while (v == u) v = distrib(gen);
        g.add_edge(u, v, weight_dist(gen));
    }

    g.print();
    g.kruskal();

    return 0;
}

Writing kruskal.cpp


In [11]:
!g++ kruskal.cpp -o kruskal
!./kruskal


Matriz de adyacencia ponderada:
. . . . . . . . 29 . . 
. . 61 . . . . . . 51 . 
. 61 . 79 . . 88 . . 89 . 
. . 79 . . . . 10 . . . 
. . . . . . . . . . . 
. . . . . . . . . . . 
. . 88 . . . . . . . . 
. . . 10 . . . . . 51 8 
29 . . . . . . . . . 52 
. 51 89 . . . . 51 . . . 
. . . . . . . 8 52 . . 

Aristas del Árbol de Expansión Mínima (Kruskal):
7 - 10 (peso 8)
3 - 7 (peso 10)
0 - 8 (peso 29)
1 - 9 (peso 51)
7 - 9 (peso 51)
8 - 10 (peso 52)
1 - 2 (peso 61)
2 - 6 (peso 88)
Peso total del MST: 350


**🌐 Algoritmo de Prim**

**Descripción**

Prim construye el MST expandiéndose desde un nodo inicial usando una cola de prioridad (en este caso un array simple por simplicidad).

**Implementación en C++**

In [13]:
%%writefile prim.cpp
#include <iostream>
#include <vector>
#include <limits>
#include <random>
using namespace std;

class Graph
{
private:
    vector<vector<int>> adj_matrix;

public:
    Graph(int n)
    {
        adj_matrix = vector<vector<int>>(n, vector<int>(n, -1));
    }

    void add_edge(int u, int v, int weight)
    {
        if (u == v)
            return; // evitar bucles
        adj_matrix[u][v] = weight;
        adj_matrix[v][u] = weight;
    }

    bool has_edge(int u, int v)
    {
        return adj_matrix[u][v] != -1;
    }

    void print()
    {
        cout << "Matriz de adyacencia ponderada:\n";
        int n = adj_matrix.size();
        for (int i = 0; i < n; i++)
        {
            for (int j = 0; j < n; j++)
            {
                if (adj_matrix[i][j] == -1)
                    cout << ". ";
                else
                    cout << adj_matrix[i][j] << " ";
            }
            cout << endl;
        }
    }

    void prim()
    {
        int n = adj_matrix.size();
        vector<int> parent(n, -1);                      // Almacena el MST
        vector<int> key(n, numeric_limits<int>::max()); // Coste mínimo para conectar
        vector<bool> inMST(n, false);                   // Nodo incluido en el MST

        key[0] = 0; // Empezamos desde el nodo 0

        for (int count = 0; count < n - 1; ++count)
        {
            // Encontrar el nodo u con menor key[] no incluido aún
            int u = -1;
            int min_key = numeric_limits<int>::max();
            for (int v = 0; v < n; ++v)
            {
                if (!inMST[v] && key[v] < min_key)
                {
                    min_key = key[v];
                    u = v;
                }
            }

            if (u == -1)
                break; // grafo no conexo

            inMST[u] = true;

            // Actualizar claves y padres de los nodos adyacentes
            for (int v = 0; v < n; ++v)
            {
                if (adj_matrix[u][v] != -1 && !inMST[v] && adj_matrix[u][v] < key[v])
                {
                    key[v] = adj_matrix[u][v];
                    parent[v] = u;
                }
            }
        }

        cout << "\nAristas del Árbol de Expansión Mínima (Prim):\n";
        int total_weight = 0;
        for (int i = 1; i < n; ++i)
        {
            if (parent[i] != -1)
            {
                cout << parent[i] << " - " << i << " (peso " << adj_matrix[i][parent[i]] << ")\n";
                total_weight += adj_matrix[i][parent[i]];
            }
        }
        cout << "Peso total del MST: " << total_weight << endl;
    }
};

int main()
{
    int n = 11;
    Graph g(n);

    // Generador de aristas aleatorias (con peso entre 1 y 20)
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> node_dist(0, n - 1);
    uniform_int_distribution<> weight_dist(1, 20);

    int edges_added = 0;
    while (edges_added < 20)
    {
        int u = node_dist(gen);
        int v = node_dist(gen);
        int w = weight_dist(gen);
        if (u != v && !g.has_edge(u, v))
        {
            g.add_edge(u, v, w);
            edges_added++;
        }
    }

    g.print();
    g.prim();

    return 0;
}

Writing prim.cpp


In [14]:
!g++ prim.cpp -o prim
!./prim


Matriz de adyacencia ponderada:
. 12 . . 9 . . 4 . . . 
12 . . . 20 16 . . . . . 
. . . . 5 . 4 8 18 18 2 
. . . . . 17 6 . 9 3 . 
9 20 5 . . . . . 20 . . 
. 16 . 17 . . 9 . 14 . . 
. . 4 6 . 9 . . 13 . . 
4 . 8 . . . . . . . . 
. . 18 9 20 14 13 . . . 15 
. . 18 3 . . . . . . . 
. . 2 . . . . . 15 . . 

Aristas del Árbol de Expansión Mínima (Prim):
0 - 1 (peso 12)
7 - 2 (peso 8)
6 - 3 (peso 6)
2 - 4 (peso 5)
6 - 5 (peso 9)
2 - 6 (peso 4)
0 - 7 (peso 4)
3 - 8 (peso 9)
3 - 9 (peso 3)
2 - 10 (peso 2)
Peso total del MST: 62
